In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import math
import statistics
from scipy.stats import norm
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn
import matplotlib.mlab as mlab
import scipy

In [4]:
class HistoricalVaR:
    
    def __init__(self, tickers, list_weights, initial_portfolio, time):
        self.tickers = tickers
        self.list_weights = list_weights
        self.initial_investment = initial_portfolio
        self.time = time
        
        """chain of method calls"""
        dict_assets = self.data_grabber(tickers)
        assets, df_returns, mean_returns = self.returns(dict_assets)
        df_portfolio = self.portfolio(dict_assets)
        dict_assets, array_weights = self.weights(dict_assets, list_weights)
        df_portfolio, df_returns, series_portfolio_returns = self.portfolio_return(df_portfolio, df_returns, array_weights)
        display(df_portfolio, df_returns)
        df_cov = self.covariance_matrix(dict_assets, df_returns)
        portfolio_variance = self.portfolio_variance_calculator(df_cov, array_weights)
        std = self.portfolio_std(portfolio_variance, time)
        Pret = self.portfolio_mean_return(mean_returns, array_weights, time)
        df_returns = self.add_portfolio_returns_to_df_returns(df_returns, series_portfolio_returns)
        dict_VaR = self.historicalVaR(df_returns, initial_portfolio, time)
        dict_CVaR = self.historicalCVaR(df_returns, dict_VaR, initial_portfolio, time)
    
    def hour_wiper(df):
        
        df = df.reset_index()
        df["Date"] = pd.to_datetime(df["Date"]).dt.date
        df = df.set_index('Date')
        
        return df

    def data_grabber(self, tickers):
        #Function designed to take any ticker and grab annual data
        dict_assets = {}
        
        for x in tickers:
            try: 
                
                ticker = yf.Ticker(x)
                ticker_data = ticker.history(period = "1y", interval = "1d")
                ticker_data = HistoricalVaR.hour_wiper(ticker_data)
                
                if not ticker_data.empty:
                    dict_assets[x] = ticker_data
                else:
                    print(f"The associated data for {x} is empty")
            except Exception as e:
                print(f"There was an error in fetching {x} data")
                
        return dict_assets

    def returns(self, dict_assets):
        #Calculates daily returns of each asset in the portfolio and puts them into a seperate dataframe
        for x in dict_assets:
            df_common = dict_assets[x]    
            df_common["Returns"] = df_common["Close"].pct_change()
            df_common = df_common.dropna()        
            dict_assets[x] = df_common
        
        dict_returns = {}    
        for x in dict_assets:
            df_common = dict_assets[x]
            series_returns = df_common["Returns"]
            dict_returns[x] = series_returns
            
        df_returns = pd.concat(dict_returns.values(), axis = 1, keys=dict_returns)
        mean_returns = df_returns.mean()
        
        return dict_assets, df_returns, mean_returns

    def weights(self, dict_assets, list_weights):
        #Takes weights and inserts them into dataframes of each asset as a column 
        
        list_counter = 0
        if isinstance(list_weights, list):
            for x in list_weights:
                x = float(x)
                x = x/100
                list_weights[list_counter] = x
                list_counter += 1
            array_weights = np.array(list_weights)
        else:
            raise TypeError("the expected data structure for the weights is a list")
        
        if np.sum(array_weights) == 1:
            pass
        else:
            print(f"The sum of your weights does not equal to 1")
        
        weights_counter = 0
        if  len(array_weights) == len(dict_assets):
            for key in dict_assets:
                df_common = dict_assets[key]
                df_common["Weights"] = list_weights[weights_counter]
                df_common = df_common.dropna()
                dict_assets[key] = df_common
                weights_counter += 1
        else:
            print(f"The number of assets and weights do not match up, please try again")
        
        return dict_assets, array_weights

    def portfolio(self, dict_assets):
        #Assembles the portfolio into a singular dataframe
        df_portfolio = pd.concat(dict_assets.values(), axis = 1, keys=dict_assets.keys())
        df_portfolio = df_portfolio.reset_index()

        return df_portfolio
        
    def portfolio_return(self, df_portfolio, df_returns, array_weights):
        #Calculates the daily returns of the portfolio given the weights
            
        series_portfolio_return = df_returns.dot(array_weights)
        df_portfolio["Portfolio Returns"] = series_portfolio_return
        display(series_portfolio_return)
        return df_portfolio, df_returns, series_portfolio_return
        
    def covariance_matrix(self, dict_assets, df_returns):
        #Finds the covariance between the returns of each asset in the portfolio        
        df_cov = df_returns.cov()
        
        return df_cov
    
    def portfolio_variance_calculator(self, df_cov, array_weights):
        #Find portfolio variance given covariance matrix and weights of the portfolio
        array_portfolio_variance = np.dot(array_weights.T, np.dot(df_cov, array_weights))
        float_portfolio_variance = float(array_portfolio_variance)
        return float_portfolio_variance  
        
    def portfolio_std(self, float_portfolio_variance, time):
        #Multiply by the square root of time to annualize the volatility
        float_portfolio_std = (math.sqrt(float_portfolio_variance))*np.sqrt(time)
        
        return float_portfolio_std
        
    def portfolio_mean_return(self, mean_returns, array_weights, time):
        #Multiply by the number of trading days to annualize average portfolio return
        P_returns = np.sum(mean_returns*array_weights)*time

        
        return P_returns

    def add_portfolio_returns_to_df_returns(self, df_returns, series_portfolio_returns):
        df_returns["Portfolio Returns"] = series_portfolio_returns
        return df_returns

    def historicalVaR(self, returns, initial_portfolio, time):
        """
        Read in a pandas dataframe of returns / a pandas series of returns
        Output the percentile of the distribution at the given alpha confidence level
        """
        list_alpha = [0.9, 0.95, 0.975, 0.99]
        dict_PreVaR = {}
        dict_PostVaR = {}
        
        if isinstance(returns, pd.Series):
            for x in list_alpha:
                VaR = np.percentile(returns, (1-x)*100)
                dict_PreVaR[x] = round((VaR * np.sqrt(time)) * initial_portfolio, 2) * -1
                dict_PostVaR[x] = VaR
        elif isinstance(returns, pd.DataFrame):
            for x in list_alpha:
                VaR = returns.apply(lambda col: np.percentile(col, (1-x)*100))
                dict_PreVaR[x] = round((VaR * np.sqrt(time)) * initial_portfolio, 2) * -1
                dict_PostVaR[x] = VaR
        
        for x in dict_PreVaR:
            dict_PreVaR[x] = (dict_PreVaR[x])["Portfolio Returns"]
            
        for x in dict_PostVaR:
            dict_PostVaR[x] = (dict_PostVaR[x])["Portfolio Returns"]
        
        display(f"The VaR with associated confidence levels for the portfolio is: \n {dict_PreVaR}")
        
        return dict_PostVaR

    def historicalCVaR(self, returns, dict_VaR, initial_portfolio, time):
        """
        Read in a pandas dataframe of returns / a pandas series of returns
        Output the CVaR for dataframe / series
        """
        list_alpha = [0.9, 0.95, 0.975, 0.99]
        dict_CVaR = {}

        if isinstance(returns, pd.Series):
            for x in list_alpha:
                belowVaR = returns <= dict_VaR[x]
                CVaR = returns[belowVaR].mean()
                dict_CVaR[x] = round((CVaR * np.sqrt(time)) * initial_portfolio, 2) * -1
            
            for x in dict_CVaR:
                dict_CVaR[x] = (dict_CVaR[x])["Portfolio Returns"]
            display(f"The CVaR with associated confidence levels for the portfolio is: \n {dict_CVaR}")
           
            return dict_CVaR

        # A passed user-defined-function will be passed a Series for evaluation.
        elif isinstance(returns, pd.DataFrame):
            series_portfolio_return = returns["Portfolio Returns"]
            for x in list_alpha:
                belowVaR = series_portfolio_return <= dict_VaR[x]
                CVaR = returns[belowVaR].mean()
                dict_CVaR[x] = round((CVaR * np.sqrt(time)) * initial_portfolio, 2) * -1
            for x in dict_CVaR:
                dict_CVaR[x] = (dict_CVaR[x])["Portfolio Returns"]
            display(f"The CVaR with associated confidence levels for the portfolio is: \n {dict_CVaR}")
            
            return dict_CVaR

        else:
            raise TypeError("Expected returns to be dataframe or series") 



In [5]:
tickers = ["TSLA", "AAPL"]
list_weights = [50,50]
time = 252
initial_portfolio = 100000
portfolio1 = HistoricalVaR(tickers, list_weights, initial_portfolio, time)

Date
2023-02-21   -0.039599
2023-02-22    0.010289
2023-02-23    0.004657
2023-02-24   -0.021845
2023-02-27    0.031425
                ...   
2024-02-12   -0.018553
2024-02-13   -0.016560
2024-02-14    0.010338
2024-02-15    0.030319
2024-02-16   -0.005462
Length: 250, dtype: float64

Date        TSLA                                                 \
                       Open        High         Low       Close     Volume   
0    2023-02-21  204.990005  209.710007  197.220001  197.369995  180018600   
1    2023-02-22  197.929993  201.990005  191.779999  200.860001  191828500   
2    2023-02-23  203.910004  205.139999  196.330002  202.070007  146360000   
3    2023-02-24  196.330002  197.669998  192.800003  196.880005  142228100   
4    2023-02-27  202.029999  209.419998  201.259995  207.630005  161028300   
..          ...         ...         ...         ...         ...        ...   
245  2024-02-12  192.110001  194.729996  187.279999  188.130005   95498600   
246  2024-02-13  183.990005  187.259995  182.110001  184.020004   86759500   
247  2024-02-14  185.300003  188.889999  183.350006  188.710007   81203000   
248  2024-02-15  189.160004  200.880005  188.860001  200.449997  120831800   
249  2024-02-16  202.059998  203.169998  197.399994  199.949997  111173600   

                                            AAPL                          \
    Dividends Stock Splits   Returns        Open        High         Low   
0         0.0          0.0 -0.052518  149.402519  150.496685  147.622030   
1         0.0          0.0  0.017683  148.079573  149.153841  146.378661   
2         0.0          0.0  0.006024  149.293105  149.541777  146.458246   
3         0.0          0.0 -0.025684  146.328931  146.408508  144.946312   
4         0.0          0.0  0.054602  146.925767  148.378007  146.667138   
..        ...          ...       ...         ...         ...         ...   
245       0.0          0.0 -0.028104  188.419998  188.669998  186.789993   
246       0.0          0.0 -0.021847  185.770004  186.210007  183.509995   
247       0.0          0.0  0.025486  185.320007  185.529999  182.440002   
248       0.0          0.0  0.062212  183.550003  184.490005  181.350006   
249       0.0          0.0 -0.002494  183.419998  184.850006  181.669998   

                                                           Portfolio Returns  
          Close    Volume Dividends Stock Splits   Returns                    
0    147.691650  58867200       0.0          0.0 -0.026680               NaN  
1    148.119370  51011300       0.0          0.0  0.002896               NaN  
2    148.606766  48394200       0.0          0.0  0.003291               NaN  
3    145.931061  55469600       0.0          0.0 -0.018005               NaN  
4    147.134644  44998500       0.0          0.0  0.008248               NaN  
..          ...       ...       ...          ...       ...               ...  
245  187.149994  41781900       0.0          0.0 -0.009002               NaN  
246  185.039993  56529500       0.0          0.0 -0.011274               NaN  
247  184.149994  54630500       0.0          0.0 -0.004810               NaN  
248  183.860001  65434500       0.0          0.0 -0.001575               NaN  
249  182.309998  49701400       0.0          0.0 -0.008430               NaN  

[250 rows x 18 columns]

,TSLA,AAPL
Date,,
2023-02-21,-0.052518,-0.026680
2023-02-22,0.017683,0.002896
2023-02-23,0.006024,0.003291
2023-02-24,-0.025684,-0.018005
2023-02-27,0.054602,0.008248
...,...,...
2024-02-12,-0.028104,-0.009002
2024-02-13,-0.021847,-0.011274
2024-02-14,0.025486,-0.004810


'The VaR with associated confidence levels for the portfolio is: \n {0.9: 33457.52, 0.95: 43428.48, 0.975: 53496.87, 0.99: 78829.4}'

'The CVaR with associated confidence levels for the portfolio is: \n {0.9: 49848.52, 0.95: 60672.26, 0.975: 73171.48, 0.99: 88294.42}'